In [2]:
!pip install cirq
!pip install mitiq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.8/598.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.6 MB/s eta 0:00:00
INFO: pip is looking a

In [3]:
from mitiq import benchmarks

circuit = benchmarks.generate_rb_circuits(n_qubits=2, num_cliffords=2)[0]
circuit

0: ───Y^-0.5───X^-0.5───Y^0.5───@───X^-0.5───Y^-0.5───Y^-0.5───X^-0.5───@───Y^0.5────@───Y────────X^0.5───X^0───@───Y^0.5────@───Y────────X^0.5───
                                │                                       │            │                          │            │
1: ───Y^0.5────X^-0.5───────────@───Y^0.5────Y^0.5────X^0.5─────────────@───X^-0.5───@───Y^-0.5───X^0.5───Y^0───@───X^-0.5───@───Y^-0.5───────────

In [7]:
from cirq import DensityMatrixSimulator, depolarize
from mitiq import Executor

def execute(circuit, noise_level=0.1):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    executed with depolarizing noise acting on two-qubit gates.
    """
    # Replace with code based on your frontend and backend.
    noisy_circuit = cirq.Circuit()
    for op in circuit.all_operations():
        noisy_circuit.append(op)
        # Add depolarizing noise after two-qubit gates
        if len(op.qubits) == 2:
            noisy_circuit.append(depolarize(p=noise_level, n_qubits=2)(*op.qubits))

    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[0, 0].real

executor = Executor(execute)

In [11]:
from functools import partial
import numpy as np
from mitiq import zne
import cirq


# Random local folding applied to two-qubit gates with a seeded random state
random_state = np.random.RandomState(0)
noise_scaling_function = partial(
    zne.scaling.fold_gates_at_random,
    fidelities = {"single": 1.0},  # Avoid folding single-qubit gates
    random_state=random_state,  # Useful to get reproducible results
)
# Exponential fit with scale factors [1, 2, 3], assuming an infinite-noise limit of 0.5.
factory = zne.inference.ExpFactory(scale_factors=[1, 2, 3], asymptote=0.25)

zne_value = zne.execute_with_zne(
    circuit=circuit,
    executor=executor,
    observable=None,
    factory=factory,
    scale_noise=noise_scaling_function,
    num_to_average=3,
)

zne_value

0.9992711307909649

In [16]:
from mitiq import ddd

rule = ddd.rules.xyxy
ideal_value = execute(circuit, noise_level=0.0)

In [17]:
mitigated_result = ddd.execute_with_ddd(
    circuit=circuit,
    executor=execute,
    rule=rule,
)

In [18]:
from mitiq.ddd import insert_ddd_sequences
ixix_circ = insert_ddd_sequences(circuit, rule)
print(ixix_circ)

0: ───Y^-0.5───X^-0.5───Y^0.5───@───X^-0.5───Y^-0.5───Y^-0.5───X^-0.5───@───Y^0.5────@───Y────────X^0.5───X^0───@───Y^0.5────@───Y────────X^0.5───
                                │                                       │            │                          │            │
1: ───Y^0.5────X^-0.5───────────@───Y^0.5────Y^0.5────X^0.5─────────────@───X^-0.5───@───Y^-0.5───X^0.5───Y^0───@───X^-0.5───@───Y^-0.5───────────


In [19]:
print(f"Error with mitigation (DDD): {abs(ideal_value - mitigated_result) :.3}")
EWM=abs(ideal_value - mitigated_result)

Error with mitigation (DDD): 0.323
